<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/batched_solvers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd
from enum import Enum

In [0]:
"""Solve a batch of QPs.
            This function solves a batch of QPs, each optimizing over
            `nz` variables and having `nineq` inequality constraints
            and `neq` equality constraints.
            The optimization problem for each instance in the batch
            (dropping indexing from the notation) is of the form
                \hat z =   argmin_z 1/2 z^T Q z + p^T z
                        subject to Gz <= h
                                    Az  = b
            where Q \in S^{nz,nz},
                S^{nz,nz} is the set of all positive semi-definite matrices,
                p \in R^{nz}
                G \in R^{nineq,nz}
                h \in R^{nineq}
                A \in R^{neq,nz}
                b \in R^{neq}
            These parameters should all be passed to this function as
            Variable- or Parameter-wrapped Tensors.
            (See torch.autograd.Variable and torch.nn.parameter.Parameter)
            If you want to solve a batch of QPs where `nz`, `nineq` and `neq`
            are the same, but some of the contents differ across the
            minibatch, you can pass in tensors in the standard way
            where the first dimension indicates the batch example.
            This can be done with some or all of the coefficients.
            You do not need to add an extra dimension to coefficients
            that will not change across all of the minibatch examples.
            This function is able to infer such cases.
            If you don't want to use any equality or inequality constraints,
            you can set the appropriate values to:
                e = Variable(torch.Tensor())
            Parameters:
            Q:  A (nBatch, nz, nz) or (nz, nz) Tensor.
            p:  A (nBatch, nz) or (nz) Tensor.
            G:  A (nBatch, nineq, nz) or (nineq, nz) Tensor.
            h:  A (nBatch, nineq) or (nineq) Tensor.
            A:  A (nBatch, neq, nz) or (neq, nz) Tensor.
            b:  A (nBatch, neq) or (neq) Tensor.
            Returns: \hat z: a (nBatch, nz) Tensor.
            """

"Solve a batch of QPs.\n            This function solves a batch of QPs, each optimizing over\n            `nz` variables and having `nineq` inequality constraints\n            and `neq` equality constraints.\n            The optimization problem for each instance in the batch\n            (dropping indexing from the notation) is of the form\n                \\hat z =   argmin_z 1/2 z^T Q z + p^T z\n                        subject to Gz <= h\n                                    Az  = b\n            where Q \\in S^{nz,nz},\n                S^{nz,nz} is the set of all positive semi-definite matrices,\n                p \\in R^{nz}\n                G \\in R^{nineq,nz}\n                h \\in R^{nineq}\n                A \\in R^{neq,nz}\n                b \\in R^{neq}\n            These parameters should all be passed to this function as\n            Variable- or Parameter-wrapped Tensors.\n            (See torch.autograd.Variable and torch.nn.parameter.Parameter)\n            If you want 

In [0]:
nbatch=2
nBatch=nbatch
nz=2
nineq=2
neq=1

In [0]:
Q=torch.tensor([4,1,1,2]).view(nz,nz)
p=torch.tensor([1,1]).view(nz)
G=torch.tensor([-1,0,0,-1]).view(nineq,nz)
h=torch.tensor([0,0]).view(nineq)
A=torch.tensor([1,1]).view(neq,nz)
b=torch.tensor([1]).view(neq)

In [0]:
#to do: extract dimensions from problem parameters + check/add batch dimension
Q=torch.tensor([[4,1,1,2],[6,2,2,2]]).view(nbatch,nz,nz).type(torch.DoubleTensor)
p=torch.tensor([[1,1],[1,6]]).view(nbatch,nz).type(torch.DoubleTensor)
G=torch.tensor([[-1,0,0,-1],[-1,0,0,-1]]).view(nbatch,nineq,nz).type(torch.DoubleTensor)
h=torch.tensor([[0,0],[0,0]]).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor([[1,1],[2,3]]).view(nbatch,neq,nz).type(torch.DoubleTensor)
b=torch.tensor([[1],[4]]).view(nbatch,neq).type(torch.DoubleTensor)

In [0]:
#check if Q is psd:
for i in range(nbatch):
  e,_=torch.eig(Q[i])
  if not torch.all(e[:,0]>0):
    raise RuntimeError("Q is not PSD")

In [0]:
def lu_hack(x):
    data, pivots = x.lu(pivot=not x.is_cuda)
    if x.is_cuda:
        if x.ndimension() == 2:
            pivots = torch.arange(1, 1+x.size(0)).int().cuda()
        elif x.ndimension() == 3:
            pivots = torch.arange(
                1, 1+x.size(1),
            ).unsqueeze(0).repeat(x.size(0), 1).int().cuda()
        else:
            assert False
    return (data, pivots)

In [0]:
class KKTSolvers(Enum):
    LU_FULL = 1
    LU_PARTIAL = 2
    IR_UNOPT = 3

In [0]:
def get_step(v, dv):
    a = -v / dv
    a[dv > 0] = max(1.0, a.max())
    return a.min(1)[0].squeeze()


In [0]:
rx=p
rs=torch.zeros(nBatch, nineq).type_as(Q)
rz=-h
ry= -b if b is not None else None
D = torch.eye(nineq).repeat(nBatch, 1, 1).type_as(Q)
H_ = torch.zeros(nBatch, nz + nineq, nz + nineq).type_as(Q)
H_[:, :nz, :nz] = Q
H_[:, -nineq:, -nineq:] = D
if neq > 0:
  A_ = torch.cat([torch.cat([G, torch.eye(nineq).type_as(Q).repeat(nBatch, 1, 1)], 2),torch.cat([A, torch.zeros(nBatch, neq, nineq).type_as(Q)], 2)], 1)
  g_ = torch.cat([rx, rs], 1)
  h_ = torch.cat([rz, ry], 1)
else:
  A_ = torch.cat([G, torch.eye(nineq).type_as(Q)], 1)
  g_ = torch.cat([rx, rs], 1)
  h_ = rz
H_  
A_

tensor([[[-1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1.],
         [ 1.,  1.,  0.,  0.]],

        [[-1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1.],
         [ 2.,  3.,  0.,  0.]]], dtype=torch.float64)

In [0]:
# H=torch.zeros(nbatch,nz+nineq+neq,nz+nineq+neq).type_as(Q)
# H[:,:nz,:nz]=Q
# H[:,:nz,nz:nz+nineq]=torch.transpose(G,2,1)
# H[:,:nz,nz+nineq:nz+nineq+neq]=torch.transpose(A,2,1)
# H[:,nz:nz+nineq,:nz]=G
# H[:,nz:nz+nineq,nz:nz+nineq]=-1*torch.eye(nineq).type_as(Q)
# H[:,nz:nz+nineq,nz+nineq:nz+nineq+neq]=torch.zeros(nbatch,nineq,neq).type_as(Q)
# H[:,nz+nineq:nz+nineq+neq,:nz]=A
# H[:,nz+nineq:nz+nineq+neq,nz:nz+nineq]=torch.zeros(nbatch,neq,nineq).type_as(Q)
# H[:,nz+nineq:nz+nineq+neq,nz+nineq:nz+nineq+neq]=torch.zeros(nbatch,neq,neq).type_as(Q)
# F=torch.cat((-p,h,b), dim=1).unsqueeze(2)

In [0]:
G_T=torch.transpose(G,dim0=2,dim1=1)
A_T=torch.transpose(A,dim0=2,dim1=1)
R1=torch.cat((Q,G_T,A_T), dim=2)
R2=torch.cat((G,-1*torch.eye(nineq).type_as(Q).repeat(nBatch, 1, 1),torch.zeros(nbatch,nineq,neq).type_as(Q)), dim=2)
R3=torch.cat((A,torch.zeros(nbatch,neq,nineq).type_as(Q),torch.zeros(nbatch,neq,neq).type_as(Q)), dim=2)
H=torch.cat((R1,R2,R3),dim=1)
F=torch.cat((-p,h,b), dim=1).unsqueeze(2)

In [0]:
G_T=torch.transpose(G,dim0=2,dim1=1)
A_T=torch.transpose(A,dim0=2,dim1=1)
R1=torch.cat((Q,torch.zeros(nbatch,nz,nineq).type_as(Q),G_T,A_T), dim=2)
R2=torch.cat((torch.zeros(nbatch,nineq,nz).type_as(Q),D,torch.eye(nineq).type_as(Q).repeat(nbatch,1,1),torch.zeros(nbatch,nineq,neq).type_as(Q)),dim=2)
R3=torch.cat((G,torch.eye(nineq).type_as(Q).repeat(nBatch, 1, 1),torch.zeros(nbatch,nineq,nineq+neq).type_as(Q)), dim=2)
R4=torch.cat((A,torch.zeros(nbatch,neq,2*nineq+neq).type_as(Q)), dim=2)
H=torch.cat((R1,R2,R3,R4),dim=1)
F=torch.cat((-p,torch.zeros(nbatch,nineq).type_as(Q),h,b), dim=1).unsqueeze(2)
#H

In [0]:
H_lu,H_piv= lu_hack(H)
initial=F.lu_solve(H_lu,H_piv)
# initial

In [0]:
x=initial[:,:nz,:]
z=initial[:,nz:nz+nineq,:]
y=initial[:,nz+nineq:nz+nineq+neq,:]
s=-z
print(x,s,z,y)

tensor([[[0.3333],
         [0.6667]],

        [[0.5294],
         [0.9804]]], dtype=torch.float64) tensor([[[0.3333],
         [0.6667]],

        [[0.5294],
         [0.9804]]], dtype=torch.float64) tensor([[[-0.3333],
         [-0.6667]],

        [[-0.5294],
         [-0.9804]]], dtype=torch.float64) tensor([[[-3.3333]],

        [[-3.3333]]], dtype=torch.float64)


In [0]:
def factor_solve_kkt(Q, D, G, A, rx, rs, rz, ry):
    nineq, nz, neq, nBatch = get_sizes(G, A)

    H_ = torch.zeros(nBatch, nz + nineq, nz + nineq).type_as(Q)
    H_[:, :nz, :nz] = Q
    H_[:, -nineq:, -nineq:] = D
    if neq > 0:
        A_ = torch.cat([torch.cat([G, torch.eye(nineq).type_as(Q).repeat(nBatch, 1, 1)], 2),
                        torch.cat([A, torch.zeros(nBatch, neq, nineq).type_as(Q)], 2)], 1)
        g_ = torch.cat([rx, rs], 1)
        h_ = torch.cat([rz, ry], 1)
    else:
        A_ = torch.cat([G, torch.eye(nineq).type_as(Q)], 1)
        g_ = torch.cat([rx, rs], 1)
        h_ = rz

    H_LU = lu_hack(H_)

    invH_A_ = A_.transpose(1, 2).lu_solve(*H_LU)
    invH_g_ = g_.unsqueeze(2).lu_solve(*H_LU).squeeze(2)

    S_ = torch.bmm(A_, invH_A_)
    S_LU = lu_hack(S_)
    t_ = torch.bmm(invH_g_.unsqueeze(1), A_.transpose(1, 2)).squeeze(1) - h_
    w_ = -t_.unsqueeze(2).lu_solve(*S_LU).squeeze(2)
    t_ = -g_ - w_.unsqueeze(1).bmm(A_).squeeze()
    v_ = t_.unsqueeze(2).lu_solve(*H_LU).squeeze(2)

    dx = v_[:, :nz]
    ds = v_[:, nz:]
    dz = w_[:, :nineq]
    dy = w_[:, nineq:] if neq > 0 else None

    return dx, ds, dz, dy


In [0]:
def get_sizes(G, A=None):
    if G.dim() == 2:
        nineq, nz = G.size()
        nBatch = 1
    elif G.dim() == 3:
        nBatch, nineq, nz = G.size()
    if A is not None:
        neq = A.size(1) if A.nelement() > 0 else 0
    else:
        neq = None
    # nBatch = batchedTensor.size(0) if batchedTensor is not None else None
    return nineq, nz, neq, nBatch

In [0]:
def bdiag(d):
    nBatch, sz = d.size()
    D = torch.zeros(nBatch, sz, sz).type_as(d)
    I = torch.eye(sz).repeat(nBatch, 1, 1).type_as(d).bool()
    D[I] = d.squeeze().view(-1)
    return D


In [0]:
def forward(Q, p, G, h, A, b, eps=1e-12, verbose=1, notImprovedLim=3,
            maxIter=5, solver=KKTSolvers.LU_FULL):

    nineq, nz, neq, nBatch = get_sizes(G, A)

    # Find initial values
    if solver == KKTSolvers.LU_FULL:
        D = torch.eye(nineq).repeat(nBatch, 1, 1).type_as(Q)
        print(D)
        x, s, z, y = factor_solve_kkt(
            Q, D, G, A, p,
            torch.zeros(nBatch, nineq).type_as(Q),
            -h, -b if b is not None else None)
    else:
        assert False

  
    best = {'resids': None, 'x': None, 'z': None, 's': None, 'y': None}
    nNotImproved = 0
    print("\nInitialized as, x:{},s:{},y:{},z:{},".format(
        x,s,y,z
    ))
    for i in range(maxIter):
        # affine scaling direction
        rx = (torch.bmm(y.unsqueeze(1), A).squeeze(1) if neq > 0 else 0.) + \
            torch.bmm(z.unsqueeze(1), G).squeeze(1) + \
            torch.bmm(x.unsqueeze(1), Q.transpose(1, 2)).squeeze(1) + \
            p
        rs = z
        rz = torch.bmm(x.unsqueeze(1), G.transpose(1, 2)).squeeze(1) + s - h
        ry = torch.bmm(x.unsqueeze(1), A.transpose(
            1, 2)).squeeze(1) - b if neq > 0 else 0.0

        mu = torch.abs((s * z).sum(1).squeeze() / nineq)
        # print('\nrx: {}, rz: {}, ry: {},'.format(
        #     rx,rz,ry
        # ))

        z_resid = torch.norm(rz, 2, 1).squeeze()
        y_resid = torch.norm(ry, 2, 1).squeeze() if neq > 0 else 0
        pri_resid = y_resid + z_resid
        dual_resid = torch.norm(rx, 2, 1).squeeze()
        resids = pri_resid + dual_resid #+ nineq * mu

        d = z / s
        # try:
        #     factor_kkt(S_LU, R, d)
        # except:
        #     return best['x'], best['y'], best['z'], best['s']

        if verbose == 1:
            print('iter: {}, pri_resid: {:.5e}, dual_resid: {:.5e}, mu: {:.5e}'.format(
                i, pri_resid.mean(), dual_resid.mean(), mu.mean()))
            print(x)
        
        if solver == KKTSolvers.LU_FULL:
            D = bdiag(d)
            dx_aff, ds_aff, dz_aff, dy_aff = factor_solve_kkt(
                Q, D, G, A, rx, rs, rz, ry)
    
        else:
            assert False
        dx = dx_aff 
        ds = ds_aff 
        dz = dz_aff 
        dy = dy_aff 

        alpha = torch.min(0.999 * torch.min(get_step(z, dz),
                                            get_step(s, ds)),
                          torch.ones(nBatch).type_as(Q))
        alpha_nineq = alpha.repeat(nineq, 1).t()
        alpha_neq = alpha.repeat(neq, 1).t() if neq > 0 else None
        alpha_nz = alpha.repeat(nz, 1).t()
        
        x += alpha_nz * dx
        s += alpha_nineq * ds
        z += alpha_nineq * dz
        y = y + alpha_neq * dy if neq > 0 else None

    return best['x'], best['y'], best['z'], best['s']


In [0]:
x,_,_,_=forward(Q,p,G,h,A,b)


Initialized as, x:tensor([[0.3333, 0.6667],
        [0.5294, 0.9804]], dtype=torch.float64),s:tensor([[0.3333, 0.6667],
        [0.5294, 0.9804]], dtype=torch.float64),y:tensor([[-3.3333],
        [-3.3333]], dtype=torch.float64),z:tensor([[-0.3333, -0.6667],
        [-0.5294, -0.9804]], dtype=torch.float64),
iter: 0, pri_resid: 4.01964e-16, dual_resid: 0.00000e+00, mu: 4.49250e-01
tensor([[0.3333, 0.6667],
        [0.5294, 0.9804]], dtype=torch.float64)
iter: 1, pri_resid: 2.27170e-16, dual_resid: 6.92343e-16, mu: 2.98874e-02
tensor([[0.1667, 0.8333],
        [0.4847, 1.0102]], dtype=torch.float64)
iter: 2, pri_resid: 1.24127e-16, dual_resid: 1.20220e-15, mu: 1.11071e-02
tensor([[0.2708, 0.7292],
        [0.5003, 0.9998]], dtype=torch.float64)
iter: 3, pri_resid: 5.55112e-17, dual_resid: 6.66134e-16, mu: 6.22983e-04
tensor([[0.2524, 0.7476],
        [0.5000, 1.0000]], dtype=torch.float64)
iter: 4, pri_resid: 1.17575e-16, dual_resid: 6.66134e-16, mu: 8.44852e-06
tensor([[0.2500, 0.750

AttributeError: ignored

In [0]:
x[0].T@Q[0]@x[0]
x[0]

tensor([0.2500, 0.7500], dtype=torch.float64)

In [0]:
for i in nbatch:
  qpth_optimum=(0.5*np.sum((Q@x[i])**2)) + q.T @ qpth_solution